This document demonstrates the making, training, saving, loading, and usage of a sklearn-compliant CGCNN model.

In [1]:
import os
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'/home/zulissi/software/adamwr')
import numpy as np
import cgcnn
#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


## Load the dataset as mongo docs

In [2]:
import random
import pickle

#Load a selection of documents
docs = pickle.load(open('/home/zulissi/software/cgcnn_sklearn/CO_docs.pkl','rb'))
random.seed(42)
random.shuffle(docs)
docs = [doc for doc in docs if -3<doc['energy']<1.0]
docs = docs[:1000]

## Get the size of the features from the data transformer, to be used in setting up the net model

In [3]:
from torch.utils.data import Dataset, DataLoader
import mongo
from cgcnn.data import StructureData, ListDataset, StructureDataTransformer
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler


SDT = StructureDataTransformer(atom_init_loc='/home/zulissi/software/cgcnn_sklearn/atom_init.json',
                              max_num_nbr=12,
                               step=0.2,
                              radius=1,
                              use_tag=True,
                              use_fixed_info=False,
                              use_distance=True)

SDT_out = SDT.transform(docs)

structures = SDT_out[0]

#Settings necessary to build the model (since they are size of vectors as inputs)
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

import multiprocess as mp
from sklearn.model_selection import ShuffleSplit

SDT_out = SDT.transform(docs)

with mp.Pool(4) as pool:
    SDT_list = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))

100%|██████████| 1000/1000 [04:19<00:00,  3.86it/s]


In [4]:
with open('distance_all_docs.pkl','wb') as fhandle:
    pickle.dump(SDT_list,fhandle)

## CGCNN model with skorch to make it sklearn compliant

In [5]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it
from cgcnn.data import collate_pool
from skorch import NeuralNetRegressor
from cgcnn.model import CrystalGraphConvNet
import torch
from cgcnn.data import MergeDataset
import skorch.callbacks.base


cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training
class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()


## Example converting all the documents up front

In [6]:
#Make the target list
target_list = np.array([doc['energy'] for doc in docs]).reshape(-1,1)

## Shuffle and Split

In [7]:
from sklearn.model_selection import train_test_split

SDT_training, SDT_test, target_training, target_test = train_test_split(SDT_list, target_list, test_size=0.2)


## Fit the model

In [8]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler
from adamw import AdamW
from cosine_scheduler import CosineLRWithRestarts

train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)

# warm restart scheduling from https://arxiv.org/pdf/1711.05101.pdf
LR_schedule = LRScheduler(CosineLRWithRestarts, batch_size=214, epoch_size=len(SDT_training), restart_period=10, t_mult=1.2)

net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    batch_size=214,
    module__classification=False,
    lr=0.0056,
    max_epochs=100, #188
    module__atom_fea_len=46,
    module__h_fea_len=1,
    module__n_conv=8,
    module__n_h=1,
    module__visualization=True, #Switch to per-atom energy mode
    optimizer=AdamW,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
#     criterion=torch.nn.MSELoss,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)



In [9]:
net.initialize()
net.fit(SDT_training,target_training)

Re-initializing module because the following parameters were re-set: atom_fea_len, classification, h_fea_len, n_conv, n_h, nbr_fea_len, orig_atom_fea_len, visualization.
Re-initializing optimizer because the following parameters were re-set: .
  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.6824        0.6249     +  0.9599
      2        0.4913        0.5982     +  0.8826
      3        0.3927        0.5843     +  0.8825
      4        0.3489        0.5739     +  0.8861
      5        0.3427        0.5609     +  0.8865
      6        0.3188        0.5418     +  0.8874
      7        0.3116        0.5172     +  0.8863
      8        0.3003        0.4936     +  0.8857
      9        0.2954        0.4687     +  0.8880
     10        0.2931        0.4464     +  0.8865
     11        0.3008        0.4133     +  0.8860
     12        0.2909        0.3968     +  0.8860
     13        0.2770        0.3733     +  0.8861
     1

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=CrystalGraphConvNet(
    (embedding): Linear(in_features=99, out_features=46, bias=True)
    (convs): ModuleList(
      (0): ConvLayer(
        (fc_full): Linear(in_features=98, out_features=92, bias=True)
        (sigmoid): Sigmoid()
        (softplus1): LeakyReLU(negative_slope=0.01)
        (bn1): BatchNorm1d(92, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (softplus2): LeakyReLU(negative_slope=0.01)
      )
      (1): ConvLayer(
        (fc_full): Linear(in_features=98, out_features=92, bias=True)
        (sigmoid): Sigmoid()
        (softplus1): LeakyReLU(negative_slope=0.01)
        (bn1): BatchNorm1d(92, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (softplus2): LeakyReLU

Add a hook to save the per-atom visualization each time the net is called

In [10]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

net.module_.visualization_layer.register_forward_hook(get_activation('visualization_layer'))


Save 5 atoms objects with the calculated per-atom energy as the "charge" attribute.  

In [15]:
index=0
for sdt,doc in zip(SDT_list[0:20],docs[0:20]):
    net.predict([sdt])
    energies = np.array(activation['visualization_layer']).reshape((-1))
    atoms = mongo.make_atoms_from_doc(doc)
    print(doc['results']['energy'])
    #energies=energies*(1-atoms.get_tags())+atoms.get_tags()*np.mean(energies)
    atoms.set_initial_charges(energies)
    atoms.write('%d.traj'%(index))
    index+=1

-1.1073901300000042
-0.01227288999998244
-0.2209785799999846
-0.3102291400000379
-0.02101047999999217
-0.4718952000000112
-0.7153526599999989
-1.4915358300000232
0.36590288999998144
-1.950037039999975
-0.06121053999999582
-0.7279071600000062
-0.2920729799999915
-0.151564590000012
0.227957599999991
-1.5148751800000095
-0.6102481800000046
-1.089113139999986
-0.1325224099999982
-1.4740079699999935


In [14]:
activation['visualization_layer']

tensor([[-0.7427],
        [-1.1437],
        [-1.5398],
        [-1.3790],
        [-1.4278],
        [-1.7802],
        [-1.4174],
        [-1.5122],
        [-1.4333],
        [-0.8792],
        [-1.0983],
        [-1.4074],
        [-1.5057],
        [-1.8026],
        [-2.1501],
        [-2.4727],
        [-2.4548],
        [-2.1757],
        [-4.5324],
        [-2.0303],
        [-1.5243],
        [-1.2144],
        [-1.4278],
        [-1.7801],
        [-1.4174],
        [-1.5117],
        [-1.4540],
        [-0.8784],
        [-1.0666],
        [-1.4035],
        [-1.5058],
        [-1.6550],
        [-1.8995],
        [-2.4358],
        [-2.3782],
        [-2.1807],
        [-5.0775],
        [-2.5906]], device='cuda:0')

In [12]:
dsfhjjkgh32515

NameError: name 'dsfhjjkgh32515' is not defined